In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from nlp.models import Tfidf, TopicModelSVD
from nlp.util import *
from wrangling.cast import CastingPool

from pymongo import MongoClient
from sklearn.cluster import KMeans


cpool = CastingPool()


## Grab two events
event_a = cpool.db.event.find_one({'title': 'Nipplegate'})
event_b = cpool.db.event.find_one({'title': 'Clinton-Lewinsky Scandal'})

In [13]:
## Grab two sets of roles
a_roles = cpool.get_roles(event_a['_id'])
b_roles = cpool.get_roles(event_b['_id'])

In [3]:
## Grab two casting pools
a_cstng_pool = cpool.get_potential_cast(event_a['_id'])
b_cstng_pool = cpool.get_potential_cast(event_b['_id'])


In [5]:
## Topic Model Cast A, Cast B

docs_a = [p['stemmed_wiki_doc'] for p in a_cstng_pool]
doc_ids_a = [p['_id'] for p in a_cstng_pool]

model_a = Tfidf(docs_a, doc_ids_a)
topic_model_a = TopicModelSVD(model_a.model, model_a.cv_tfidf)


docs_b = [p['stemmed_wiki_doc'] for p in b_cstng_pool]
doc_ids_b = [p['_id'] for p in b_cstng_pool]

model_b = Tfidf(docs_b, doc_ids_b)
topic_model_b = TopicModelSVD(model_b.model, model_b.cv_tfidf)

In [14]:
## Topic Model Roles A, Roles B

a_roles_topic_mdl = get_SVD_model(a_roles)
b_roles_topic_mdl = get_SVD_model(b_roles)

In [15]:
## Cluster
cpool_a_kmeans = KMeans(n_clusters=10)
cpool_b_kmeans = KMeans(n_clusters=10)

cpool_a_kmeans.fit(model_a.model)
cpool_b_kmeans.fit(model_b.model)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
## Match

In [20]:
a_roles_tfidf, a_roles_topics = get_SVD_model(a_roles)

In [25]:
a_roles_tfidf.model

,aa,aaa,aaliyah,aaron,ab,abandon,abbi,abbott,abc,abcnews,...,zuniga,zurbrugg,zz,éireann,남자들,빅뱅,빅뱅家의,승리와의,인터뷰,화보
5c6f04ccfc0c4b94ecceef0b,0.001990,0.000000,0.01044,0.001398,0.000000,0.002088,0.002088,0.000000,0.004706,0.000000,...,0.004176,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5c6f04ccfc0c4b94ecceef0c,0.001944,0.001646,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002298,0.000000,...,0.000000,0.000000,0.000000,0.00204,0.00204,0.00204,0.00204,0.00204,0.00204,0.00204
5c6f04ccfc0c4b94ecceef0d,0.005258,0.000000,0.00000,0.002463,0.003678,0.000000,0.000000,0.000000,0.002072,0.000000,...,0.000000,0.000000,0.003678,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5c6f04ccfc0c4b94ecceef0e,0.009267,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021914,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5c6f04ccfc0c4b94ecceef0f,0.000874,0.000740,0.00000,0.004914,0.000000,0.000000,0.000000,0.000917,0.000000,0.000917,...,0.000000,0.000917,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [30]:
for role in a_roles:
    print (role['name'] + '\'s id is: ' + str(role['_id']))

Janet Jackson's id is: 5c6f04ccfc0c4b94ecceef0b
Justin Timberlake's id is: 5c6f04ccfc0c4b94ecceef0c
Kid Rock's id is: 5c6f04ccfc0c4b94ecceef0d
Greg Gumbel's id is: 5c6f04ccfc0c4b94ecceef0e
Tom Brady's id is: 5c6f04ccfc0c4b94ecceef0f


In [105]:
#Transform Roles A & B accoring to models of Roles:
import pandas as pd

docs_a_r = [p['stemmed_wiki_doc'] for p in a_roles]
doc_ids_a_r = [p['_id'] for p in a_roles]

a_roles_transformed = model_a.cv_tfidf.transform(docs_a_r).toarray()


#import numpy as np
#for role in a_roles:
#    cpool_a_kmeans.predict(np.array(a_roles_transformed.loc[role['_id']]).reshape(1,-1))
cpool_a_kmeans.predict(np.array(a_roles_transformed[4]).reshape(1,-1))

array([0], dtype=int32)

In [66]:
#Greg Gumbel was placed in group 9. Choose a random candidate to play him in the movie. 
candidates = []
for i in zip(cpool_a_kmeans.labels_, a_cstng_pool):
    if i[0] == 9:
        candidates.append((i[1]['name']))

print (f'{np.random.choice(candidates)} chosen to play Greg Gumbel')

Michael Lerner (actor) chosen to play Greg Gumbel


In [106]:

min_ = -np.inf
name = ''
person = None
for i in zip(cpool_a_kmeans.labels_, a_cstng_pool):
    
    #Serach the correct cluster
    if i[0] == 0:
        
        #a_roles_transformed is a role
        #model_a is the casting pool vector tfdidf
        dist = cosine_distance(np.array(a_roles_transformed[4]), model_a.model.loc[i[1]['_id']])
        
        if dist > min_:
            min_ = dist
            name = i[1]['name']
            person = i[1]
            print(name + ' ' + str(dist))
            
print (name)

Steve McQueen 0.05403310811370573
Sylvester Stallone 0.0602013971787115
Woody Allen 0.09141866336359379
Harrison Ford 0.09475745401787523
Matt Damon 0.10181566982759678
Will Smith 0.10246418948405499
Joaquin Phoenix 0.10612969815905478
Jesse Eisenberg 0.12324032232016979
Andrew Garfield 0.12451911863453054
Timothée Chalamet 0.13608998159731928
Timothée Chalamet


In [86]:
type(model_a.model.loc[i[1]['_id']])

pandas.core.series.Series

In [87]:
from bson.objectid import ObjectId
greg_gumbel = ObjectId('5c6f04ccfc0c4b94ecceef0e')
a_roles_tfidf.model.loc[greg_gumbel]

pandas.core.series.Series

In [93]:
def cosine_distance(u, v):
    """
    Returns the cosine of the angle between vectors v and u. This is equal to
    u.v / |u||v|.
    """
    return numpy.dot(u, v) / (math.sqrt(numpy.dot(u, u)) * math.sqrt(numpy.dot(v, v)))

In [19]:
from nlp.models import Tfidf, TopicModelSVD

def get_SVD_model(people):
    
    docs = [p['stemmed_wiki_doc'] for p in people]
    doc_ids = [p['_id'] for p in people]
    
    model = Tfidf(docs, doc_ids)
    topic_model = TopicModelSVD(model.model, model.cv_tfidf)
    
    return (model, topic_model)

In [3]:
cast = cpool.get_casting_pool()
docs = [p['stemmed_wiki_doc'] for p in cast]
doc_ids = [p['_id'] for p in cast]



model = Tfidf(docs, doc_ids)
topic_model = TopicModelSVD(model.model, model.cv_tfidf)

In [5]:
topic_model = TopicModelSVD(model.model, model.cv_tfidf)

In [58]:
from sklearn.cluster import KMeans
means = KMeans(n_clusters=15, random_state=51)

In [ ]:
means = KMeans(n_clusters=15, random_state=51)

In [59]:
means.fit(model.model)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=3, random_state=51, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [60]:
means.inertia_

738.669601596358

In [39]:
np.array(model.model).shape

(1060, 80806)

In [67]:
means.predict()

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [111]:
min_ = -np.inf
name = ''
person = None
for i in zip(cpool_a_kmeans.labels_, a_cstng_pool):
    
    #Serach the correct cluster
    #if i[0] == 0:
        
    #a_roles_transformed is a role
    #model_a is the casting pool vector tfdidf
    dist = cosine_distance(np.array(a_roles_transformed[0]), model_a.model.loc[i[1]['_id']])

    if dist > min_:
        min_ = dist
        name = i[1]['name']
        person = i[1]
        print(name + ' ' + str(dist))

print (name)

Peter O'Toole 0.12631355092771873
Richard Burton 0.1330834600937386
Laurence Olivier 0.1361747203832702
Rod Steiger 0.14307874182390548
James Earl Jones 0.14677735250988222
Woody Allen 0.14899710934935617
Jeff Bridges 0.17907671371299091
Robin Williams 0.1797301772873159
Will Smith 0.24539435597755532
Samuel L. Jackson 0.4274059203214055
Samuel L. Jackson
